In [1]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
from importlib import reload
from src.classifiers_davidjames9610.test_a.config import config_location
import importlib
import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)
import time

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands


In [3]:
# time = 20 ms
# fs = 16000 = 1/t
# windows samples = 20 * 10^-3 * 1/1600
# = 0.2 * 1600 = 800

In [3]:
import pickle
import d_classifiers_to_results
reload(d_classifiers_to_results)

sls = pickle.load(open(config.samples_labels,'rb'))

In [5]:
features = pickle.load(open(config.features,'rb'))

In [4]:
classifiers = pickle.load(open(config.classifiers,'rb'))
results = {}
plot_cms = True
save_cms = True

<module 'src.misc_davidjames9610.utils' from '/Users/david/Documents/code/fhmm/v1/src/misc_davidjames9610/utils.py'>

In [5]:
classifiers

loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR40.pickle
loading mfcc_GaussNoise_SNR60.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR60.pickle
loading mfcc_GaussNoise_SNR40.pickle
loading mfcc_None_None.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR20.pickle
loading mfcc_GaussNoise_SNR10.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR10.pickle
loading mfcc_GaussNoise_SNR20.pickle


In [6]:
classifiers_gmm =
# d_classifiers_to_results.include_buffer_in_features(features, buffer_length=250, buffer_step=125)
# import pickle
# pickle.dump(features, open(config.features,'wb'))

loading GMMHMM.pickle


In [7]:
import d_classifiers_to_results
reload(d_classifiers_to_results)
import src.misc_davidjames9610.utils as utils
reload(utils)
start_time = time.time()

results['classification_annotations'] = d_classifiers_to_results.get_classification_results(
    features, classifiers, sls, config.basedir, plot_cm=plot_cms, save_plots=save_cms)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

In [18]:
import d_classifiers_to_results
reload(d_classifiers_to_results)
start_time = time.time()

results['classification_buffer'] = d_classifiers_to_results.get_classification_buff_results(
    features, classifiers, sls, config.basedir, plot_cm=plot_cms, save_plots=save_cms)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

In [ ]:
import d_classifiers_to_results
reload(d_classifiers_to_results)
start_time = time.time()

results['classification_annotations_valg'] = d_classifiers_to_results.get_classification_valg_results(
    features, classifiers, sls, config.basedir, plot_cm=plot_cms, save_plots=save_cms)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Traceback (most recent call last):
  File "/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_comm.py", line 293, in _on_run
    r = self.sock.recv(1024)
ConnectionResetError: [Errno 54] Connection reset by peer


KeyError: 'mfcc'

In [ ]:
import d_classifiers_to_results
reload(d_classifiers_to_results)
start_time = time.time()

results['classification_buffer_valg'] = d_classifiers_to_results.get_classification_valg_buffer_results(
    features, classifiers, sls, config.basedir, plot_cm=plot_cms, save_plots=save_cms)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

In [21]:
import pickle
pickle.dump(results, open(config.results,'wb'))

Directory '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/results/classification_buffer_valg' created.
saving / updating  GaussianHMM
